In [1]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub

In [2]:
mlflow.set_tracking_uri('https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow')
dagshub.init(repo_owner='Abhaysharma53', repo_name='mlops-mini-project', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\admin\Desktop\DSMP\mlops-mini-project\my-mlops-env\lib\site-packages\rich\live.py:229: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=034799c6-c4bf-4e26-b6d6-9f293674aefb&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=419b58d1f9e425399d28dc37798f85e06a41b266bbb89d5d4a59af45d1d21cf6




Accessing as Abhaysharma53

Initialized MLflow to track repo "Abhaysharma53/mlops-mini-project"

Repository Abhaysharma53/mlops-mini-project initialized!

In [13]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [14]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

In [15]:
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    text = [word for word in str(text).split() if word not in stop_words ]
    return " ".join(text)
    

In [16]:
def removing_numbers(text):
    text = ''.join([char for char in text if not char.isdigit()])
    return text

In [17]:
def lower_case(text):
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

In [18]:
def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [19]:
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [20]:
#normalize the text data
df = normalize_text(df)

In [21]:
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [22]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [23]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [24]:
# Set the experiment name
mlflow.set_experiment("Bow vs TfIdf")

2024/10/14 00:57:09 INFO mlflow.tracking.fluent: Experiment with name 'Bow vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1b8d5d4377b148df9e1788f82f1c35c9', creation_time=1728847628737, experiment_id='2', last_update_time=1728847628737, lifecycle_stage='active', name='Bow vs TfIdf', tags={}>

In [25]:
# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [28]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # Save and log the notebook
                #mlflow.log_artifact(__file__)
                # Save and log the notebook
                import os
                notebook_path = "C:\\Users\\admin\\Desktop\\DSMP\\mlops-mini-project\\mlops-mini-project\\notebooks\\exp_1- Baseline_experiment.ipynb"
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2024/10/14 01:25:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7932530120481928
Precision: 0.7833655705996132
Recall: 0.7980295566502463
F1 Score: 0.7906295754026355


2024/10/14 01:25:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with BoW at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2/runs/1115a691dedd4dd3a207f966b97a11d4.
2024/10/14 01:25:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2.
2024/10/14 01:25:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.7784090909090909
Recall: 0.8098522167487685
F1 Score: 0.7938194109126028


2024/10/14 01:26:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2/runs/87459d0133e44bb9a49a164b02cfef81.
2024/10/14 01:26:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2.
2024/10/14 01:26:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

2024/10/14 01:33:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest with TF-IDF at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2/runs/46662c48b4d5468d850f3cfd2bde73d3.
2024/10/14 01:33:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2.
2024/10/14 01:34:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.7291566265060241
Precision: 0.805668016194332
Recall: 0.5881773399014778
F1 Score: 0.6799544419134396


2024/10/14 01:34:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with BoW at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2/runs/7b0ea119b6934dbab6ab172c98255fe4.
2024/10/14 01:34:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2.
2024/10/14 01:35:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.7228915662650602
Precision: 0.8055555555555556
Recall: 0.5714285714285714
F1 Score: 0.6685878962536023


2024/10/14 01:36:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2/runs/93c4341387aa40d888df23feb481d8d0.
2024/10/14 01:36:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2.
2024/10/14 01:36:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run All Experiments at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2/runs/6efce18fadda441d8bf583b1f77cc42f.
2024/10/14 01:36:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/2.
